In [184]:
import pandas as pd
from datetime import datetime
from pandas import DataFrame
from tabulate import tabulate
from IPython.core.display import HTML
import time

In [222]:
template_head = 'backgroundColor: \'#2c343c\', title: [{{text: \'{0}\', left: \'center\', top: 20, textStyle: {{color: \'#ccc\'}}}}, {1}, {2}, {3}, {4}], tooltip: {{trigger: \'item\'}}, visualMap: {{show: false, min: 1000, max: 5000, inRange: {{colorLightness: [1, 0]}}}},'
template_sub_text = '{{subtext: \'{0}\', left: \'{1}\', top: \'{2}\', textAlign: \'center\', fontSize: 20, textStyle: {{color: \'#ccc\'}}}}'
template_series = '{{name: \'详细信息\', type: \'pie\', radius: \'20%\', center: {0}, {1}, roseType: \'radius\', label: {{color: \'rgba(255, 255, 255, 0.3)\'}}, labelLine: {{lineStyle: {{color: \'rgba(255, 255, 255, 0.3)\'}}, smooth: 0.2, length: 10, length2: 20}}, itemStyle: {{color: \'{2}\', shadowBlur: 200, shadowColor: \'rgba(0, 0, 0, 0.5)\'}}, animationType: \'scale\', animationEasing: \'elasticOut\', animationDelay: function (idx) {{return Math.random() * 200;}}}},'
template_data_set = 'data: [{0}].sort(function (a, b) {{return a.value - b.value;}})'
template_data = '{{value: {0}, name: \'{1}\', tooltip: {{formatter: \'{{b}}:&lt;br/&gt;{2}total: {{c}}kg\'}}}},'
template_item = '{0}kg × {1}&lt;br/&gt;'
define_parts = [
    {
        'part':'胸',
        'left':'25%',
        'top': '45%',
        'center':'[\'25%\', \'30%\']',
    },
    {
        'part':'背',
        'left': '75%',
        'top': '45%',
        'center':'[\'75%\', \'30%\']'
    },
    {
        'part':'手臂',
        'left': '25%',
        'top': '90%',
        'center':'[\'25%\', \'75%\']'
    },
    {
        'part':'腿',
        'left':'75%',
        'top': '90%',
        'center':'[\'75%\', \'75%\']'
    },
]
pd.set_option('max_colwidth', 2000)
pd.set_option('display.unicode.east_asian_width', True)

In [223]:
data = pd.read_csv('../datas/trains.csv', encoding='gbk')

def generate_head(date_str):
    part1 = template_sub_text.format(define_parts[0]['part'], define_parts[0]['left'], define_parts[0]['top'])
    part2 = template_sub_text.format(define_parts[1]['part'], define_parts[1]['left'], define_parts[1]['top'])
    part3 = template_sub_text.format(define_parts[2]['part'], define_parts[2]['left'], define_parts[2]['top'])
    part4 = template_sub_text.format(define_parts[3]['part'], define_parts[3]['left'], define_parts[3]['top'])
    head = template_head.format(date_str, part1, part2, part3, part4)
    return head

def left_align(df: DataFrame):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'center')])]
    )
    return left_aligned_df

def filter_pd_data(_type, raw):
    ret = {}
    for index, row in raw.iterrows():
        key = row[_type]
        if key not in ret:
            ret[key] = []
        ret[key].append(row)
    return ret
data_by_date = filter_pd_data('DATE', data)

data_formatter = {}
for key, value in data_by_date.items():
    data_set_str = ''
    center = ''
    color = ''
    df = pd.DataFrame(value)
    data_by_name = filter_pd_data('NAME', df)
    for name, row in data_by_name.items():
        total = 0
        item_str = ''
        for item in row:
            weight = item['WEIGHT']
            times = item['TIMES']
            center = item['CENTER']
            color = item['COLOR']
            total += weight * times
            item_str += template_item.format(weight, times)
        data_set_str += template_data.format(total, name, item_str)
    data_formatter[key] = template_series.format(center, template_data_set.format(data_set_str), color)
    
formatter_data = pd.DataFrame(data_formatter.items(), columns=['Date', 'Formatter'])
# formatter_data.style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
left_align(formatter_data)
# display(HTML(formatter_data.to_html(index=False, justify='left')))
# formatter_data.to_html()

,Date,Formatter
0,2021/11/25,"{name: '详细信息', type: 'pie', radius: '20%', center: ['50%', '50%'], data: [{value: 7200, name: '腿举', tooltip: {formatter: '{b}:<br/>120kg × 10<br/>140kg × 10<br/>140kg × 10<br/>160kg × 10<br/>160kg × 10<br/>total: {c}kg'}},{value: 2120, name: '直腿硬拉', tooltip: {formatter: '{b}:<br/>60kg × 8<br/>80kg × 8<br/>100kg × 3<br/>100kg × 4<br/>100kg × 3<br/>total: {c}kg'}},{value: 2980, name: '自由深蹲', tooltip: {formatter: '{b}:<br/>60kg × 8<br/>80kg × 8<br/>100kg × 3<br/>100kg × 3<br/>100kg × 3<br/>60kg × 8<br/>60kg × 8<br/>total: {c}kg'}},{value: 1632, name: '坐姿腿屈伸', tooltip: {formatter: '{b}:<br/>34kg × 12<br/>34kg × 12<br/>34kg × 12<br/>34kg × 12<br/>total: {c}kg'}},{value: 3168, name: '罗马挺', tooltip: {formatter: '{b}:<br/>66kg × 12<br/>66kg × 12<br/>66kg × 12<br/>66kg × 12<br/>total: {c}kg'}},].sort(function (a, b) {return a.value - b.value;}), roseType: 'radius', label: {color: 'rgba(255, 255, 255, 0.3)'}, labelLine: {lineStyle: {color: 'rgba(255, 255, 255, 0.3)'}, smooth: 0.2, length: 10, length2: 20}, itemStyle: {color: '#c23531', shadowBlur: 200, shadowColor: 'rgba(0, 0, 0, 0.5)'}, animationType: 'scale', animationEasing: 'elasticOut', animationDelay: function (idx) {return Math.random() * 200;}},"
1,2021/11/29,"{name: '详细信息', type: 'pie', radius: '20%', center: ['25%', '30%'], data: [{value: 2300, name: '仰卧杠铃卧推', tooltip: {formatter: '{b}:<br/>60kg × 8<br/>60kg × 8<br/>60kg × 8<br/>60kg × 6<br/>60kg × 6<br/>70kg × 2<br/>total: {c}kg'}},{value: 1980, name: '上斜器械推举', tooltip: {formatter: '{b}:<br/>30kg × 12<br/>30kg × 12<br/>30kg × 12<br/>40kg × 10<br/>50kg × 10<br/>total: {c}kg'}},{value: 1382, name: '窄距哑铃推举', tooltip: {formatter: '{b}:<br/>30kg × 12<br/>31kg × 12<br/>32kg × 10<br/>33kg × 10<br/>total: {c}kg'}},{value: 2600, name: '双杠臂屈伸', tooltip: {formatter: '{b}:<br/>65kg × 10<br/>65kg × 10<br/>65kg × 10<br/>65kg × 10<br/>total: {c}kg'}},{value: 1100, name: '站姿高位拉锁夹胸', tooltip: {formatter: '{b}:<br/>30kg × 10<br/>30kg × 10<br/>30kg × 10<br/>20kg × 10<br/>total: {c}kg'}},{value: 2925, name: '悬垂卷腹', tooltip: {formatter: '{b}:<br/>65kg × 15<br/>65kg × 15<br/>65kg × 15<br/>total: {c}kg'}},{value: 2475, name: '跪姿拉索卷腹', tooltip: {formatter: '{b}:<br/>55kg × 15<br/>55kg × 15<br/>55kg × 15<br/>total: {c}kg'}},].sort(function (a, b) {return a.value - b.value;}), roseType: 'radius', label: {color: 'rgba(255, 255, 255, 0.3)'}, labelLine: {lineStyle: {color: 'rgba(255, 255, 255, 0.3)'}, smooth: 0.2, length: 10, length2: 20}, itemStyle: {color: '#316B83', shadowBlur: 200, shadowColor: 'rgba(0, 0, 0, 0.5)'}, animationType: 'scale', animationEasing: 'elasticOut', animationDelay: function (idx) {return Math.random() * 200;}},"
2,2021/11/30,"{name: '详细信息', type: 'pie', radius: '20%', center: ['75%', '30%'], data: [{value: 2745, name: '宽握引体向上', tooltip: {formatter: '{b}:<br/>65kg × 10<br/>65kg × 8<br/>65kg × 8<br/>65kg × 7<br/>75kg × 4<br/>75kg × 4<br/>total: {c}kg'}},{value: 2400, name: '俯身杠铃划船', tooltip: {formatter: '{b}:<br/>60kg × 10<br/>60kg × 10<br/>60kg × 10<br/>60kg × 10<br/>total: {c}kg'}},{value: 4250, name: '坐姿拉索划船', tooltip: {formatter: '{b}:<br/>65kg × 10<br/>75kg × 12<br/>75kg × 12<br/>75kg × 12<br/>75kg × 12<br/>total: {c}kg'}},{value: 2800, name: '高位宽握下拉', tooltip: {formatter: '{b}:<br/>65kg × 10<br/>75kg × 10<br/>75kg × 10<br/>65kg × 10<br/>total: {c}kg'}},{value: 2000, name: '器械坐姿划船', tooltip: {formatter: '{b}:<br/>40kg × 10<br/>40kg × 10<br/>40kg × 10<br/>40kg × 10<br/>20kg × 20<br/>total: {c}kg'}},{value: 750, name: '直臂下压', tooltip: {formatter: '{b}:<br/>15kg × 10<br/>15kg × 10<br/>20kg × 10<br/>25kg × 10<br/>total: {c}kg'}},{value: 2250, name: '坐姿卷腹', tooltip: {formatter: '{b}:<br/>30kg × 25<br/>30kg × 25<br/>30kg × 25<br/>total: {c}kg'}},{value: 1000, name: '俄式转体', tooltip: {formatter: '{b}:<br/>10kg × 50<br/>10kg × 50<br/>total: {c}kg'}},].sort(function (a, b) {return a.value - b.value;}), roseType: 'radius', label: {color: 'rgba(255, 255, 255, 0.3)'}, labelLine: {lineStyle: {color: 'rgba(255,

### Generate Head

In [201]:
head = generate_head('2021/11/29 - 2021/12/01')
head

"backgroundColor: '#2c343c', title: [{text: '2021/11/29 - 2021/12/01', left: 'center', top: 20, textStyle: {color: '#ccc'}}, {subtext: '胸', left: '25%', top: '45%', textAlign: 'center', fontSize: 20, textStyle: {color: '#ccc'}}, {subtext: '背', left: '75%', top: '45%', textAlign: 'center', fontSize: 20, textStyle: {color: '#ccc'}}, {subtext: '手臂', left: '25%', top: '90%', textAlign: 'center', fontSize: 20, textStyle: {color: '#ccc'}}, {subtext: '腿', left: '75%', top: '90%', textAlign: 'center', fontSize: 20, textStyle: {color: '#ccc'}}], tooltip: {trigger: 'item'}, visualMap: {show: false, min: 1000, max: 5000, inRange: {colorLightness: [1, 0]}},"

### Generate Data by Date

In [ ]:
formatter_data[formatter_data['Date'] == time.strftime('%Y/%m/%d', time.localtime(time.time()))]['Formatter']